In [1]:
import pandas as pd

In [2]:
from util.preprocess_util import prepare_data

[nltk_data] Downloading package punkt to /home/iliav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_json("./data/data.jsonl", lines=True)

In [4]:
test_data = pd.read_json("./data/test_data.jsonl", lines=True)

In [5]:
data.shape

(8638, 4)

In [6]:
train_data = data[~(data.question_id.isin(list(test_data.question_id)))].copy()
train_data.shape

(7258, 4)

In [7]:
%%time
full_train_data = prepare_data(train_data)

CPU times: user 22.9 s, sys: 76.1 ms, total: 23 s
Wall time: 23 s


/home/iliav/Documents/workspaces/counter-ai/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/iliav/Documents/workspaces/counter-ai/venv/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [8]:
full_train_data

,question_id,question,target,answer,tokenized_answer,tokenized_question,stemmed_answer,creativity,n_unique_words,vocabulary_size,stealing_strength,stealing_frequency,answer_length,sentences,sentence_length_mean,sentence_length_std
2,292357,What is the difference between 'git pull' and ...,0,"In the simplest terms, git pull does a git fet...","[in, the, simplest, terms,, git, pull, doe, a,...","[what, is, the, differ, between, git, pull, an...","in the simplest terms, git pull doe a git fetc...",0.525862,128,0.551724,1.098612,0.060625,7.336286,"[In the simplest terms, git pull does a git fe...",4.936434,4.393313
3,477816,Which JSON content type do I use?\n\nThere are...,1,The most widely used and supported JSON conten...,"[the, most, wide, use, and, support, json, con...","[which, json, content, type, do, i, use?, ther...",the most wide use and support json content typ...,0.384615,40,0.615385,1.386294,0.202941,5.929589,[The most widely used and supported JSON conte...,4.830977,3.629901
4,5767325,How can I remove a specific item from an array...,0,Find the index of the array element you want t...,"[find, the, index, of, the, array, element, yo...","[how, can, i, remov, a, specif, item, from, an...",find the index of the array element you want t...,0.471616,119,0.519651,0.693147,0.000000,7.346010,[Find the index of the array element you want ...,5.395574,5.486239
8,244777,Can comments be used in JSON?\n\nCan I use com...,0,No.\nJSON is data-only. If you include a comme...,"[no., json, is, data-only., if, you, includ, a...","[can, comment, be, use, in, json?, can, i, use...",no. json is data-only. if you includ a comment...,0.676471,99,0.727941,0.693147,0.000000,7.003065,"[No., JSON is data-only., If you include a com...",5.058064,5.163695
9,79923,What and where are the stack and heap?\n\n\nWh...,1,The stack and heap are two areas of memory use...,"[the, stack, and, heap, are, two, area, of, me...","[what, and, where, are, the, stack, and, heap?...",the stack and heap are two area of memori use ...,0.285714,68,0.359788,1.386294,0.234840,6.967909,[The stack and heap are two areas of memory us...,4.768800,3.618351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633,5143769,How do I check if a given Python string is a s...,1,"Yes, Python has a built-in method called the ""...","[yes,, python, has, a, built-in, method, call,...","[how, do, i, check, if, a, given, python, stri...","yes, python has a built-in method call the ""in...",0.453333,49,0.653333,1.609438,0.550831,6.120297,"[Yes, Python has a built-in method called the ...",5.427150,4.255613
8634,5878697,How do I remove blank elements from an array?\...,0,"There are many ways to do this, one is reject\...","[there, are, mani, way, to, do, this,, one, is...","[how, do, i, remov, blank, element, from, an, ...","there are mani way to do this, one is reject n...",0.656716,54,0.805970,0.693147,0.000000,5.908083,"[There are many ways to do this, one is reject...",4.516339,2.964839
8635,7054972,Java system properties and environment variabl...,1,System properties (System.getProperties()) are...,"[system, properti, (system.getproperties()), a...","[java, system, properti, and, environ, variabl...",system properti (system.getproperties()) are a...,0.439024,45,0.548780,1.098612,0.085158,6.278521,[System properties (System.getProperties()) ar...,4.890349,3.534317
8636,7785374,How to prevent column break within an element?...,0,The correct way to do this is with the break-i...,"[the, correct, way, to, do, this, is, with, th...","[how, to, prevent, column, break, within, an, ...",the correct way to do this is with the break-i...,0.375405,155,0.501618,1.945910,0.327213,7.682943,[The correct way to do this is with the break-...,5.890723,5.761569


In [9]:
%%time
test_data = prepare_data(test_data, train=False)

CPU times: user 7.91 s, sys: 30.2 ms, total: 7.94 s
Wall time: 7.94 s


In [10]:
selected_features = ['answer_length', 'creativity', 'stealing_strength', 'sentence_length_mean', 'vocabulary_size']

X_train = full_train_data[['stemmed_answer'] + selected_features]
y_train = full_train_data.target
X_test = test_data[['stemmed_answer'] + selected_features]
y_test = test_data.target

In [11]:
from sklearn.model_selection import cross_val_score
from train import forest_pipeline
from sklearn.metrics import f1_score, accuracy_score

In [12]:
pipe = forest_pipeline()
cross_val_score(pipe, X_train, y_train, scoring='f1').mean()

0.9188108664511263

In [13]:
pipe.fit(X_train, y_train)
f1_score(y_test, pipe.predict(X_test))

0.8702397743300424

In [14]:
accuracy_score(y_test[y_test==0], pipe.predict(X_test)[y_test==0])

0.8922863099374566

In [15]:
accuracy_score(y_test[y_test==1], pipe.predict(X_test)[y_test==1])

0.8527988942639945

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
confusion_matrix(pipe.predict(X_test), y_test)

array([[1284,  213],
       [ 155, 1234]])